In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.offline as po

In [2]:
#read in latest info from https://github.com/datasets/covid-19
df = pd.read_csv("https://raw.githubusercontent.com/datasets/covid-19/master/data/time-series-19-covid-combined.csv")
df.Date = pd.to_datetime(df.Date,format="%Y-%m-%d")
#create population column for later
df["Population"] = np.NaN
df.head()

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths,Population
0,2020-01-22,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,NaN
1,2020-01-23,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,NaN
2,2020-01-24,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,NaN
3,2020-01-25,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,NaN
4,2020-01-26,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,NaN


In [3]:
ref = pd.read_csv("https://raw.githubusercontent.com/datasets/covid-19/master/data/reference.csv")
ref.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
0,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.93911,67.709953,Afghanistan,38928341.0
1,8,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.15330,20.168300,Albania,2877800.0
2,12,DZ,DZA,12.0,NaN,NaN,NaN,Algeria,28.03390,1.659600,Algeria,43851043.0
3,20,AD,AND,20.0,NaN,NaN,NaN,Andorra,42.50630,1.521800,Andorra,77265.0
4,24,AO,AGO,24.0,NaN,NaN,NaN,Angola,-11.20270,17.873900,Angola,32866268.0


In [14]:
df[(df["Country/Region"] == "China") & (df["Province/State"].isnull())]

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths,Population


In [16]:
ref[(ref["Country_Region"] == "China") & (ref["Province_State"].isnull())]

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
232,156,CN,CHN,156.0,NaN,NaN,NaN,China,30.5928,114.3055,China,1.404676e+09


In [18]:
tmp = df[df.Date == df.Date.max()].copy()[["Country/Region","Province/State"]]

In [61]:
data = set()
df2 = df.copy()
for row in tmp.itertuples():
    if row[2] is np.NaN:
        pop = ref[(ref["Country_Region"] == row[1])]["Population"].values[0]
        df2.loc[df2["Country/Region"] == row[1],"Population"] = pop
    else:
        try:
            pop = ref[(ref["Country_Region"] == row[1]) & (ref["Province_State"] == row[2])]["Population"].values[0]
            df2.loc[(df2["Country/Region"] == row[1]) & (df2["Province/State"] == row[2]),"Population"] = pop
        except:
            continue
df2[(df2.Population.isnull()) & (df2.Date == df2.Date.max())]
df2.drop(df2[(df2["Country/Region"] == "Diamond Princess") | (df2["Province/State"] == "Diamond Princess") ].index,axis=0,inplace=True)
df2.drop(df2[(df2["Country/Region"] == "MS Zaandam") | (df2["Province/State"] == "Grand Princess") ].index,axis=0,inplace=True)
df2[(df2.Population.isnull()) & (df2.Date == df2.Date.max())]

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths,Population
4523,2020-04-17,Canada,Recovery aggregated,56.1304,-106.3468,NaN,10545.0,NaN,NaN


In [66]:
df2["DPM"] = round(((1000000 / df2.Population) * df2.Deaths),3)
df2["CPM"] = round(((1000000 / df2.Population) * df2.Confirmed),3)
df2["RPM"] = round(((1000000 / df2.Population) * df2.Recovered),3)
df2["Mort"] = df2.Deaths / df2.Confirmed * 100.0
tmp = df2[df2.Date == df2.Date.max()]
tmp = tmp.sort_values(by=["Mort"])[-10:]
tmp

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths,Population,DPM,CPM,RPM,Mort
13049,2020-04-17,Italy,NaN,43.0000,12.0000,172434.0,42727.0,22745.0,60461828.0,376.188,2851.948,706.677,13.190554
21314,2020-04-17,United Kingdom,NaN,55.3781,-3.4360,108692.0,0.0,14576.0,67886004.0,214.713,1601.096,0.000,13.410371
15224,2020-04-17,Mauritania,NaN,21.0079,10.9408,7.0,2.0,1.0,4649660.0,0.215,1.505,0.430,14.285714
2087,2020-04-17,Belgium,NaN,50.8333,4.0000,36138.0,7961.0,5163.0,11589616.0,445.485,3118.136,686.908,14.286900
260,2020-04-17,Algeria,NaN,28.0339,1.6596,2418.0,846.0,364.0,43851043.0,8.301,55.141,19.293,15.053763
16529,2020-04-17,Netherlands,Sint Maarten,18.0425,-63.0548,57.0,12.0,9.0,42882.0,209.878,1329.229,279.838,15.789474
1652,2020-04-17,Bahamas,NaN,25.0343,-77.3963,54.0,6.0,9.0,393248.0,22.886,137.318,15.258,16.666667
19835,2020-04-17,Sudan,NaN,12.8628,30.2176,33.0,4.0,6.0,43849269.0,0.137,0.753,0.091,18.181818
3131,2020-04-17,Burundi,NaN,-3.3731,29.9189,5.0,0.0,1.0,11890781.0,0.084,0.420,0.000,20.000000
4523,2020-04-17,Canada,Recovery aggregated,56.1304,-106.3468,NaN,10545.0,NaN,NaN,NaN,NaN,NaN,NaN
